In [1]:
import os
import sys
import numpy as np
import pickle
from scipy.stats.stats import pearsonr


module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
a = os.path.join(module_path, "strands_qsr_lib\qsr_lib\src3")
sys.path.append(a)

import project
# Need to add this import to load class
from project import Project
import config
import progress_learner
from rl import block_movement_env

In [2]:
from importlib import reload
reload(block_movement_env)

<module 'rl.block_movement_env' from 'D:\\git\\learn-to-perform\\rl\\block_movement_env.py'>

In [3]:
prefix = os.path.join( "..", "experiments", "human_evaluation_2d" )

#### Loading projects from files

In [4]:
import tensorflow as tf

sess =  tf.Session()

In [5]:
projects = {}
progress_estimators = {}
configs = {}

action_types = ["SlideToward", "SlideAway", "SlideNext", "SlidePast", "SlideAround"]

for project_name in action_types:
    configs[project_name] = config.Config()
    if project_name == 'SlideNext':
        configs[project_name].n_input = 8
        
    print ('========================================================')
    print ('Load for action type = ' + project_name)
    p_name = project_name.lower() + "_project.proj"

    projects[project_name] = project.Project.load(os.path.join('..', 'learned_models', p_name))
    
    with tf.variable_scope("model") as scope:
        print('-------- Load progress model ---------')
        progress_estimators[project_name] = progress_learner.EventProgressEstimator(is_training=True, name = projects[project_name].name, config = configs[project_name])  

Load for action type = SlideToward
-------- Load progress model ---------
self.inputs.shape = (?, 20, 200)  after linear layer
output.shape = (30, 200) after LSTM
self.output.shape = (30,) after linear
self._targets.shape = (?,) 
Load for action type = SlideAway
-------- Load progress model ---------
self.inputs.shape = (?, 20, 200)  after linear layer
output.shape = (30, 200) after LSTM
self.output.shape = (30,) after linear
self._targets.shape = (?,) 
Load for action type = SlideNext
-------- Load progress model ---------
self.inputs.shape = (?, 20, 200)  after linear layer
output.shape = (30, 200) after LSTM
self.output.shape = (30,) after linear
self._targets.shape = (?,) 
Load for action type = SlidePast
-------- Load progress model ---------
self.inputs.shape = (?, 20, 200)  after linear layer
output.shape = (30, 200) after LSTM
self.output.shape = (30,) after linear
self._targets.shape = (?,) 
Load for action type = SlideAround
-------- Load progress model ---------
self.inputs.

In [6]:
for project_name in action_types:
    saver = tf.train.Saver(tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES, scope='model/' + project_name))

    saver.restore(sess, os.path.join('..', 'learned_models', 'progress_' + project_name + '.mod'))

INFO:tensorflow:Restoring parameters from ..\learned_models\progress_SlideToward.mod
INFO:tensorflow:Restoring parameters from ..\learned_models\progress_SlideAway.mod
INFO:tensorflow:Restoring parameters from ..\learned_models\progress_SlideNext.mod
INFO:tensorflow:Restoring parameters from ..\learned_models\progress_SlidePast.mod
INFO:tensorflow:Restoring parameters from ..\learned_models\progress_SlideAround.mod


In [13]:
## Function to load demonstrations
def load_data_from_folder ( folder, c, p, pe, sess ):
    """
    c: Config
    p: Project
    pe: Progress_estimator
    sess: Tensorflow session
    """
    prev_demonstrations = []
    for index in range(30):
        stored_config_file = os.path.join(folder, str(index) + ".dat")
        try:
            with open(stored_config_file, 'rb') as fh:
                # need this encoding 
                stored_config = pickle.load(fh, encoding='latin-1')

                e = block_movement_env.BlockMovementEnv(c, p.speed, p.name, 
                        progress_estimator = pe, session = sess)
                e.reset_env_to_state(stored_config['start_config'], stored_config['action_storage'])

                prev_demonstrations.append(e)
        except IOError:
            pass
    return prev_demonstrations

#### Test for Slide Closer To

Using the method **test_slide_close**, we have one hyperparameter to set:
threshold. By default, it is 2 * block_size

In [23]:
import automatic_evaluator
reload(automatic_evaluator)

<module 'automatic_evaluator' from 'D:\\git\\learn-to-perform\\automatic_evaluator.py'>

In [14]:
close_demos = load_data_from_folder(os.path.join(prefix, 'SlideClose'), configs['SlideToward'], 
                                    projects['SlideToward'], progress_estimators['SlideToward'], sess)

In [24]:
for demo in close_demos:
    print (automatic_evaluator.test_slide_close(demo))

0
1
0
0
1
0
1
1
0
1
0
0
0
1
0
0
0
0
0
0
1
0
0
1
0
0
0
1
1
